# Singular Perturbation Analysis for Parabolic Partial Differential Equations
## Cristopher Arenas

In [5]:
import numpy as np
import scipy as sc
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt
from ipywidgets import interact

## Method of multiple scales

This method is used in a wide variety of problems. Tipically, there are two phisical proceses acting simultaneously using different scales.

Consider the following diffusion-advection equation, valid for a periodic domain $\theta \in [0,2\pi]$ and $t \geq 0$

\begin{align*}
\frac{\partial f(\theta,t)}{\partial t} + \frac{\partial}{\partial \theta} (\omega(\theta)\,f(\theta,t)) &= \varepsilon\frac{\partial^2f(\theta,t)}{\partial \theta^2}\\
f(\theta,0) &= F(\theta)\\
f(\theta,t) &= f(\theta+2\pi,t)
\end{align*}

where $\omega(\theta)$ is given over $0 \leq \theta \leq 2\pi$, periodic and positive. The previous equation describes advection in the periodic domain with a small diffusion, since $\varepsilon$ is a tiny value.

The method of multiple scales work with two scales. First, the advection occurs on the fast scale $\tau = t$ and the diffusion on the slow scale $T = \varepsilon\,t$. The method propose the following asymptotic expansion:

$$
f(\theta,t; \varepsilon) \,\,\,\tilde{}\,\,\, f_0(\theta,\tau,T) + \varepsilon\,f_1(\theta,\tau,T)
$$

**Numerical Example**

The previous equation is considered for the following functions $\omega(\theta)$ and $F(\theta)$.
\begin{align}
\omega(\theta) &= \sin^2{\left(\frac{\theta}{2}\right)}\\
\omega'(\theta) &= \frac{1}{2}\sin{(\theta)}\\
F(\theta) &= -\theta^2+\pi
\end{align}

In [20]:
def w(theta):
    #return (np.sin(0.5*theta))**2
    return 1
    
def wp(theta):
    #return 0.5*np.sin(theta)
    return 0
    
def F(theta):
    #return -theta*(theta-2*np.pi)
    return 5*np.sin(theta)


In [21]:
def implicit_method(N,M,eps):
    Theta = np.linspace(0,2*np.pi,N)
    T = np.linspace(0,20,M)

    dth = Theta[1]-Theta[0]
    dt = T[1]-T[0]

    A = np.zeros((N-1,N-1))
    b = np.zeros((N-1,M))

    for i in range(N-1):
        A[i,i] = 1+ wp(Theta[i])*dt + ((w(Theta[i])*dt)/dth) + 2*(eps*dt)/dth**2
        b[i,0] = F(Theta[i])
        if i==0:
            A[i,-1] = -eps*dt/dth**2  
        elif i==N-2:
            A[0,i] = 0#-w(Theta[i])*dt/dth - eps*dt/dth**2
        if i>0:
            A[i-1,i] = -eps*dt/dth**2
        if i<N-2:    
            A[i+1,i] = -w(Theta[i])*dt/dth - eps*dt/dth**2
    #print(A)
    for j in range(M-1):
        b[:,j+1] = np.linalg.solve(A,b[:,j])
    return b

In [24]:
def simulation(i,N,M,eps=0.001):
    #y1 = advection_process(N,N,1)
    y1 = implicit_method(N,M,0)
    y2 = implicit_method(N,M,0.00000001)
    y3 = implicit_method(N,M,1)
    y4 = implicit_method(N,M,10)
    Theta = np.linspace(0,2*np.pi,N)
    T = np.linspace(0,100,M)
    
    plt.plot(Theta[:-1],y1[:,i],"yo", label="eps=0")
    #plt.plot(Theta[:-1],y2[:,i],"bo", label="eps=0.1")
    #plt.plot(Theta[:-1],y3[:,i],"ro", label = "eps=1")
    #plt.plot(Theta[:-1],y4[:,i],"go", label = "eps=100")
    plt.ylim(-10,10)
    plt.grid(True)
    plt.title("t = "+str(T[i])+" [s]")
    plt.legend(loc="best")
    plt.show()
    
    

In [25]:
interact(simulation,i=(0,69),N=80,M=50)

<function __main__.simulation>

In [109]:
def advection_process(N,M,omega):
    Theta = np.linspace(0,2*np.pi,N)
    T = np.linspace(0,20,M)

    dth = Theta[1]-Theta[0]
    dt = T[1]-T[0]

    A = np.zeros((N-1,N-1))
    b = np.zeros((N-1,M))

    for i in range(N-1):
        b[i,0] = F(Theta[i])
        A[i,i] = 1+omega*(dt/dth)
        if i>0:
            A[i-1,i] = 1
        #else:
        #    A[-1,i] = 1
    #print(A)
    for j in range(M-1):
        b[:,j+1] = np.linalg.solve(A,b[:,j])
    return b

# Introduction

Commonly, exist some phenomena that can be represented as mathematical models, using differential equations. In some cases, simplificationes in the moment of making the model can reduce the difficulty of the representation, but in change, the model could not consider the effect of small factors that are occurring. 